## Import Azure ML SDK

In [1]:
# !pip install azure

^C


In [55]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

## Check  Azure SDK version

In [16]:
print(azureml.core.VERSION)

1.42.0


## Create Azure ML Workspace

In [48]:
# ws = Workspace.create(name='HR-model-managment',
#                       subscription_id="262a3c1d-7fe3-4bfb-9f90-7a5b89109336",
#                       resource_group='HRanalytics-890', 
#                       create_resource_group=False, 
#                       location='eastus'
#                       # location='southeastasia'
#                      ) 

Deploying StorageAccount with name hrmodelmstorage1a63aa454.
Deploying KeyVault with name hrmodelmkeyvault46caf1f6.
Deploying AppInsights with name hrmodelminsights496c4fa1.
Deployed AppInsights with name hrmodelminsights496c4fa1. Took 1.6 seconds.
Deployed StorageAccount with name hrmodelmstorage1a63aa454. Took 19.65 seconds.
Deployed KeyVault with name hrmodelmkeyvault46caf1f6. Took 17.96 seconds.
Deploying Workspace with name HR-model-managment.
Deployed Workspace with name HR-model-managment. Took 18.98 seconds.


## Write Configuration to local file

In [56]:
ws.write_config()

## Access from existing config file

In [57]:
ws = Workspace.from_config()

In [24]:
ws.get_details()

{'id': '/subscriptions/262a3c1d-7fe3-4bfb-9f90-7a5b89109336/resourceGroups/HRanalytics-890/providers/Microsoft.MachineLearningServices/workspaces/HR-model-managment',
 'name': 'HR-model-managment',
 'identity': {'principal_id': '536ea0d0-6ea5-4fd4-a703-0f90b016d01e',
  'tenant_id': '76a2ae5a-9f00-4f6b-95ed-5d33d77c4d61',
  'type': 'SystemAssigned'},
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {'createdByToolkit': 'sdk-v1-1.42.0'},
 'sku': 'Basic',
 'workspaceid': '5d8aa1ce-8645-4cc1-a21a-290b3709c1a1',
 'sdkTelemetryAppInsightsKey': '71fa86c2-fe23-49db-bb2e-8537edfb6ead',
 'description': '',
 'friendlyName': 'HR-model-managment',
 'containerRegistry': '/subscriptions/262a3c1d-7fe3-4bfb-9f90-7a5b89109336/resourceGroups/HRanalytics-890/providers/Microsoft.ContainerRegistry/registries/5d8aa1ce86454cc1a21a290b3709c1a1',
 'keyVault': '/subscriptions/262a3c1d-7fe3-4bfb-9f90-7a5b89109336/resourceGroups/HRanalytics-890/providers/Microsoft.Keyvault/v

## Create Azure ML Experiment

In [ ]:
exp = Experiment(workspace=ws, name='salexp')

## Start logging metrics

In [ ]:
run = exp.start_logging()
run.log("Experiment start time", str(datetime.datetime.now()))

## Train your ML model (Code)

## Freeze the model

In [ ]:
filename = 'outputs/catboost.pkl'
joblib.dump(lm, filename)

## Test Model

In [ ]:
filename = 'outputs/catboost.pkl'
loaded_model=joblib.load(filename) 
y=loaded_model.predict([[21]])[0] 
print(y)

## Log metrics to Azure ML Experiment

In [ ]:
run.log('Intercept :', lm.intercept )
run.log('Slope :', lm.coef_[0])

## End Azure ML Experiment

In [ ]:
run.log("Experiment end time", str(datetime.datetime.now()))
run.complete()

## Get Portal URL

In [ ]:
print(run.get_portal_url())

## Register the model

In [67]:
model = Model.register(model_path = "outputs/catboost.pkl",
                       model_name = "catboost",
                       tags = {"key": "1"},
                       description = "HR Prediction",
                       workspace = ws)

Registering model catboost


## Define Azure ML Deploymemt configuration

In [68]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               memory_gb=1,
                                               tags={"data": "HR","method":"sklearn"},
                                               description='Predict Stack')

## Create enviroment configuration file

In [69]:
salenv = CondaDependencies()
salenv.add_conda_package("scikit-learn")
salenv.add_conda_package("catboost")
salenv.add_conda_package("joblib")

with open("salenv.yml","w") as f:
    f.write(salenv.serialize_to_string())
with open("salenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.8.12

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
- catboost
- joblib
channels:
- anaconda
- conda-forge



## Create Azure ML Scoring file

In [70]:
%%writefile score.py

import json
import numpy as np
import os
import pickle 
import catboost
from azureml.core.model import Model
import joblib

def init():
    global model
    # retrive the path to the modelfile using model name
    model_path = Model.get_model_path("catboost")
    model =joblib.load(model_path)
    
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    #make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Overwriting score.py


## Deploy the model to Azure Container Instance

In [71]:
image_config = ContainerImage.image_configuration(execution_script="score.py",
                                                  runtime="python",
                                                  conda_file="salenv.yml")

/tmp/ipykernel_4109/919725643.py:1: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  image_config = ContainerImage.image_configuration(execution_script="score.py",


## Expose web service

In [63]:
from azureml.core import Environment
env = Environment.from_conda_specification("experiment_env", "salenv.yml")

In [64]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'my-custom-env-service'
# env = "salenv.yml"
inference_config = InferenceConfig(entry_script='score.py', environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                      name=service_name,
                      models=[model],
                      inference_config=inference_config,
                      deployment_config=aci_config,
                      overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-07-27 14:08:08+00:00 Creating Container Registry if not exists.
2022-07-27 14:08:08+00:00 Registering the environment.
2022-07-27 14:08:09+00:00 Building image..
2022-07-27 14:18:26+00:00 Generating deployment configuration.
2022-07-27 14:18:27+00:00 Submitting deployment to compute..
2022-07-27 14:18:31+00:00 Checking the status of deployment my-custom-env-service..
2022-07-27 14:23:05+00:00 Checking the status of inference endpoint my-custom-env-service.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 636fb8c0-94cd-45ef-ae44-177f7131f012
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, NameError: name 'joblib' is not defined, please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, NameError: name 'joblib' is not defined, please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 636fb8c0-94cd-45ef-ae44-177f7131f012
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, NameError: name 'joblib' is not defined, please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, NameError: name 'joblib' is not defined, please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: 636fb8c0-94cd-45ef-ae44-177f7131f012\nMore information can be found using '.get_logs()'\nError:\n{\n  \"code\": \"AciDeploymentFa

In [72]:
service = Webservice.deploy_from_model(workspace=ws,
                                       name='myservice',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_4109/2184231486.py:1: DeprecationWarning: deploy_from_model has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  service = Webservice.deploy_from_model(workspace=ws,


Creating image
Running.........................................................................................
Succeeded
Image creation operation finished for image myservice:3, operation "Succeeded"
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-07-27 14:36:06+00:00 Generating deployment configuration.
2022-07-27 14:36:06+00:00 Submitting deployment to compute.
2022-07-27 14:36:10+00:00 Checking the status of deployment myservice..
2022-07-27 14:38:16+00:00 Checking the status of inference endpoint myservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [ ]:
service.wait_for_deployment(show_output=True)

## Get the Web Service URL

In [76]:
import pandas as pd
df = pd.read_csv('Ready_to_model_TS_26july (1).csv')

In [77]:
df.head(1)

,ggid,gender,unified_grade,city,sub_bu_name,capgemini_tenure_year,age,service_line,reported_date,technology,...,Months_since_Last_promotion,supervisor_id,n1emp_id,Tenure_in_project_in_years,Marital Status,education_score,Rating_diff_from_last_year,Average_last_2_Ratings,Payposition,Leaves_takes_90 days
0,VKPDhptZjydjHsECYsBeOg==,Male,C2,Hyderabad,I&D Core BI,9.463014,42.638356,SAS,2021-01-31,SAS,...,113.556164,/5LUgpG66fPguKyddVNYFQ==,NIA0xTyiXJaAqLXv9sIOeg==,0.24915,Marr.,53.0,0.0,3.0,1.005141,0.0


In [73]:
print(service.scoring_uri)

http://e124b17b-cf07-4ea0-84f9-8f38f337f3d3.eastus.azurecontainer.io/score


In [78]:
!curl -X POST \
    -H 'Content-Type':'application/json' \
    -d '{"data":[["Male","C2","Hyderabad","I&D Core BI",9.463014,42.638356,"SAS","SAS","Niche","JOHN LEWIS PARTNERSHIP PLC",113.556164,"/5LUgpG66fPguKyddVNYFQ==","NIA0xTyiXJaAqLXv9sIOeg==",0.249150,"Marr.",53.0,0.0,3.0,1.005141,0.0]]}' \
        http://e124b17b-cf07-4ea0-84f9-8f38f337f3d3.eastus.azurecontainer.io/score

"[0]"

## Delete Workspace and clean up resources

In [46]:
ws.delete()